# A* algorithm (set-covering problem)

Copyright **`(c)`** 2023 Thomas Baracco `<s308722@studenti.polito.it>`  
[`https://github.com/baraccothomas/computational-intelligence`](https://github.com/baraccothomas/computational-intelligence)  
Free for personal or classroom use.

## Imports and constants

In [2]:
import numpy as np

from random import random
from collections import namedtuple
from functools import reduce
from queue import PriorityQueue, SimpleQueue, LifoQueue
from math import ceil


In [3]:
PROBLEM_SIZE = 100
NUM_SETS = 1000
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

State = namedtuple('State', ['taken', 'not_taken'])

## Node class

In [4]:
class Node:
    def __init__(self, state, g, h):
        self.state = state
        self.g = g
        self.h = h
    
    def f(self) -> np.int64:
        return self.g + self.h

    def __str__(self):
        return f"State: {self.state} | g = {self.g} | h = {self.h} | f = {self.f()}"
    
    def __eq__(self, other):
        return (self.state == other.state) and (self.g == other.g)

    def __ne__(self, other):
        return not (self == other)

    def __lt__(self, other):
        return (self.state < other.state) and (self.g < other.g)

    def __gt__(self, other):
        return (self.state > other.state) and (self.g > other.g)

    def __le__(self, other):
        return (self < other) or (self == other)

    def __ge__(self, other):
        return (self > other) or (self == other)

## Goal check

In [5]:
def goal_check(state):
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for i in range(PROBLEM_SIZE)]),
        )
    )

## h functions

In [6]:
def h1(state):
        return PROBLEM_SIZE - sum(
            reduce(
                np.logical_or,
                [SETS[i] for i in state.taken],
                np.array([False for _ in range(PROBLEM_SIZE)]),
            )
    )

def h2(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate

def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate

def h4(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

def h(state):
    return h4(state)

## Main code

### Goal check assert

In [12]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

### Depth First

In [215]:
frontier = SimpleQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put(state)

counter = 0
current_state = frontier.get()

while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put(new_state)
    current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

Solved in 19 steps (2 tiles)


### A star

In this algorithm we use a ```PriorityQueue``` to sort the open list of nodes by ```f```, in order to make the ```get()``` method return the node (state) with the lowest ```f```. Since this is a queue, the node (state) will be always removed from the open list. It's like it's been moved to a "closed" list without the need to instantiate it.

Every time a new node is discovered, it's added to the queue, with ```g = current_node.g + 1```, which indicates the number of sets taken in a given state, and ```h = current_node.h + h(new_state)```. The ```f``` is calculated automatically and can be called with the method ```.f()```

In [7]:
frontier = PriorityQueue()
initialState = State(set(), set(range(NUM_SETS)))
initialNode = Node(initialState, 0, h(initialState))
frontier.put((initialNode.f(), initialNode))

counter = 0
_, current_node = frontier.get()

while not goal_check(current_node.state):
    counter += 1
    for action in current_node.state[1]:
        new_state = State(
            current_node.state.taken ^ {action},
            current_node.state.not_taken ^ {action},
        )
        new_node = Node(new_state, current_node.g + 1, current_node.h + h(new_state))
        frontier.put((new_node.f(), new_node))
    
    
    
    _, current_node = frontier.get()

print(current_node.state)

print(
    f"Solved in {counter:,} steps ({len(current_node.state.taken)} tiles)"
)